**CS420 ASSIGNMENT 1 SOLUTIONS (CODE)**

`NAME: KANG CHIN SHEN`

`MATRIC ID: 01412921`

**Question 3**

In [ ]:
# Install pgmpy on Google Collab
!pip install pgmpy

In [ ]:
# Import relevant packages
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
import sys

**Create the Bayesian network**

In [ ]:
# Create a model which containts edges zof the graph
model = BayesianModel([('Smoking', 'Yellow Fingers'),
                       ('Smoking', 'Cancer'),
                       ('Solar Flare', 'Radiation'), 
                       ('Microwave', 'Radiation'),
                       ('Radiation', 'Cancer'),
                       ('Radiation', 'Skin Burn'),
                      ])

# Enter conditional probability distribution for each variable:

# Prior probability for Smoking
cpd_smoking = TabularCPD(variable='Smoking', variable_card=2, values=[[0.9], [0.1]])

# Prior probability for Solar Flare
cpd_solar_flare = TabularCPD(variable='Solar Flare', variable_card=2, values=[[0.999], [0.001]])

# Prior probability for Microwave
cpd_microwave = TabularCPD(variable='Microwave', variable_card=2, values=[[0.001], [0.999]])

# Conditional probability for P(Yellow Fingers | Smoking)
cpd_yellow_fingers = TabularCPD(variable='Yellow Fingers', 
                               variable_card=2, 
                               values = [[0.9, 0.1],
                                         [0.1, 0.9]],
                               evidence = ['Smoking'],
                               evidence_card=[2])

# Conditional probability for P(Radiation | Solar Flare, Microwave)
cpd_radiation = TabularCPD(variable='Radiation',
                           variable_card=2, 
                           values = [[0.99, 0.8, 0.7, 0.5],
                                     [0.01, 0.2, 0.3, 0.5]],
                           evidence = ['Solar Flare', 'Microwave'],
                           evidence_card=[2, 2])

# Conditional probability for P(Cancer | Smoking, Radiation)
cpd_cancer = TabularCPD(variable='Cancer',
                           variable_card=2, 
                           values = [[0.9, 0.4, 0.8, 0.1],
                                     [0.1, 0.6, 0.2, 0.9]],
                           evidence = ['Smoking', 'Radiation'],
                           evidence_card=[2, 2])

# Conditional probability for P(Skin Burn | Radiation) 
cpd_skin_burn = TabularCPD(variable='Skin Burn', 
                        variable_card=2, 
                        values = [[0.99, 0.9],
                                  [0.01, 0.1]],
                        evidence = ['Radiation'],
                        evidence_card=[2])

# Insert the nodes and conditional probability into the defined model
model.add_cpds(cpd_smoking,
               cpd_solar_flare, 
               cpd_microwave, 
               cpd_yellow_fingers,
               cpd_radiation,
               cpd_cancer,
               cpd_skin_burn)
               

**Validate Network Parameters**

In [ ]:
# Validate the Bayesian Model
print('Result of validation of constructed Bayesian Network: ' + str(model.check_model()))

In [ ]:
# Check the conditional probability associated with each node to ensure correctness

print('Prior probability for Smoking')
print(model.get_cpds('Smoking'))

print('\nPrior probability for Solar Flare')
print(model.get_cpds('Solar Flare'))

print('\nPrior probability for Microwave')
print(model.get_cpds('Microwave'))

print('\nConditional probability for P(Yellow Fingers | Smoking)')
print(model.get_cpds('Yellow Fingers'))

print('\nConditional probability for P(Radiation | Solar Flare, Microwave)')
print(model.get_cpds('Radiation'))

print('\nConditional probability for P(Cancer | Smoking, Radiation)')
print(model.get_cpds('Cancer'))

print('\nConditional probability for P(Skin Burn | Radiation)')
print(model.get_cpds('Skin Burn'))


In [ ]:
############### Inference ##################

# Import VariableElimination library
from pgmpy.inference import VariableElimination

# Set up for Variable Elimination
infer = VariableElimination(model)


In [ ]:
# Question 3 (Part 2) Compute P(Radiation | Cancer = 1)
phi_query = infer.query(['Radiation'], evidence={'Cancer':1}, joint = False)
factor = phi_query['Radiation']
print('Probability of Radiation given Cancer = 1:')
print(factor)

In [ ]:
# Question 3 (Part 3) Compute P(Cancer | Skin Burn = 1)
phi_query = infer.query(['Cancer'], evidence={'Skin Burn':1}, joint = False)
factor = phi_query['Cancer']
print('Probability of Cancer given Skin Burn = 1:')
print(factor)

In [ ]:
# Question 3 (Part 5) Compute P(Cancer = 1 | Microwave = 0)
phi_query = infer.query(['Cancer'], evidence={'Microwave':0}, joint = False)
factor = phi_query['Cancer']
print('Probability of Cancer given Microwave = 0:')
print(factor)



---



**Question 4**

In [ ]:
# Import the required libraries

%matplotlib inline
%tensorflow_version 2.x

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from keras import layers

In [ ]:
# (a) Download and load the k_mnist dataset.
(train_images, train_labels), (test_images, test_labels) = tfds.as_numpy(tfds.load(
    'kmnist',
    split=['train', 'test'],
    batch_size=-1,
    as_supervised=True,
))

In [ ]:
# Reshape the images to required dimension
train_images = train_images.reshape((60000, 28, 28))
test_images = test_images.reshape((10000, 28, 28))

In [ ]:
# Check shapes of train_images, train_labels etc
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

In [ ]:
# Assign name to corresponding classes
class_names = ['o', 'ki', 'su', 'tsu', 'na', 'ha', 'ma', 'ya', 're', 'wo']

# Show first 25 training images below
plt.figure(figsize=(10,10))

for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])

In [ ]:
# (b) Create ANN with :
# 1 input layer
# 3 hidden layers, with 128, 128, and 64 nodes each, and all using relu activation function
# This configuration achieves overall accuracy of >80% on the testing dataset after training 

model = tf.keras.Sequential()
model.add(layers.Flatten(input_shape=(28, 28)))
model.add(layers.Dense(128, activation=tf.nn.relu))
model.add(layers.Dense(128, activation=tf.nn.relu))
model.add(layers.Dense(64, activation=tf.nn.relu))

In [ ]:
# (c) Create 1 output layer with:
# size of 10
# softmax activation function
model.add(layers.Dense(10, activation=tf.nn.softmax))

In [ ]:
# (d) Compile the neural network with:
# cross entropy loss function
# Adamax optimizer with learning rate of 1×10^-3
model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the neural network
# Run the stochastic gradient descent for batch size : 16 and epochs : 6  
model.fit(train_images, train_labels, batch_size=16, epochs=6)

In [ ]:
# Evaluate the result of the model applied on the test images
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

In [ ]:
# Get all predictions for test data
predictions = model.predict(test_images)

In [ ]:
# Prediction visualization
# Correct predictions are highlighted in green
# Incorrect predictions are highlighted in red

plt.figure(figsize=(10,10))

for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_images[i], cmap=plt.cm.binary)
    predicted_label = np.argmax(predictions[i])
    true_label = test_labels[i]
    if predicted_label == true_label:
      color = 'green'
    else:
      color = 'red'
    plt.xlabel("{} ({})".format(class_names[predicted_label], 
                                  class_names[true_label]),
                                  color=color)

In [ ]:
# (e) Plot the average training error on the y-axis vs the epoch number 

# Simulate a similar training environment
eval_model = tf.keras.Sequential()
eval_model.add(layers.Flatten(input_shape=(28, 28)))
eval_model.add(layers.Dense(128, activation=tf.nn.relu))
eval_model.add(layers.Dense(128, activation=tf.nn.relu))
eval_model.add(layers.Dense(64, activation=tf.nn.relu))
eval_model.add(layers.Dense(10, activation=tf.nn.softmax))

eval_model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

hist = eval_model.fit(train_images, train_labels, batch_size=16, epochs=6)

In [ ]:
# Extract the training metrics for evaluation
hist_df = pd.DataFrame(hist.history)

epoch = [1,2,3,4,5,6]
hist_df['epoch no'] = epoch 
hist_df = hist_df.set_index('epoch no')

hist_df

In [ ]:
# Plot the result 
SAMPLE_NO = 60000

hist_arr = hist_df.to_numpy()

x = np.array([1,2,3,4,5,6])


y = np.array([
     hist_arr[0][0],
     hist_arr[1][0],
     hist_arr[2][0],
     hist_arr[3][0],
     hist_arr[4][0],
     hist_arr[5][0],
    ])


y = y / SAMPLE_NO

plt.figure(figsize=(10,10))
plt.plot(x,y,color='red',marker='o')
plt.xlabel('Epoch Number')
plt.ylabel('Avarage Training Error')
plt.title("Average Training Error against Epoch Number")
plt.grid(True)
plt.show()


In [ ]:
# (f) final accuracy of different classes and overall accuracy on the testing data

predicted_classes = [0,0,0,0,0,0,0,0,0,0]
expected_classes = [0,0,0,0,0,0,0,0,0,0]

for i in range(10000):
  exp_label = int(test_labels[i])
  pre_label = int(np.argmax(predictions[i]))

  if exp_label == pre_label:
    predicted_classes[pre_label] += 1

  expected_classes[exp_label] += 1

print('-----Final accuracy for different classes-----')

for i in range(10):
  perc_val = "{:.2f}".format(predicted_classes[i]/expected_classes[i] * 100)
  print("For label " + str(i) + " ('" + str(class_names[i]) + 
        "') : Accuracy is " + str(perc_val) + "%\n")
  
print('\n\n-----Final accuracy for test dataset-----')
print('overall accuracy: ' + str("{:.2f}".format(test_acc * 100)) + '\n\n')



---



**Question 5**

In [ ]:
# Import the required libraries
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import keras.applications.mobilenet_v2

from keras import layers
from keras.datasets import cifar10

### Import Dataset

In [ ]:
# Load training data, labels; and testing data and their true labels
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
print ('Training data seize:', train_images.shape)
print ('Test data size', test_images.shape)

# Normalize pixel values between -1 and 1
train_images = train_images / 127.5 - 1 
test_images = test_images / 127.5 - 1 

# Class names for different classes
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# Testing the GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

### Visualize dataset

In [ ]:
# Show first 25 training images below
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i][0]])

In [ ]:
# Check the shapes of train_images, train_labels, etc
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

### Resize images for use with MobileNetV2

In [ ]:
# Upsize all training and testing images to 96x96 for use with mobile net

# Minimum size requried for mobileNetV2
minSize = 96 

# Initialize a numpy array resized_train_images to store all the resized training images
resized_train_images = np.zeros((50000, minSize, minSize, 3), dtype=np.float32)

# Resize the training images
for i in range(50000):
  resized_train_images[i] = cv2.resize(train_images[i], dsize=(minSize, minSize), interpolation=cv2.INTER_AREA)
 
# Initialize a numpy array resized_test_images to store all the resized test images
resized_test_images = np.zeros((10000, minSize, minSize, 3), dtype=np.float32)

# Resize the test images
for i in range(10000):
  resized_test_images[i] = cv2.resize(test_images[i], dsize=(minSize, minSize), interpolation=cv2.INTER_AREA)

In [ ]:
print(resized_train_images.shape)
print(train_images.shape)

### Download MobileNetV2 model



In [ ]:
# (a) Download the pre-trained MobileNetV2 network from Keras Applications
IMG_SHAPE = (96, 96, 3)

# (b) Remove the final output layer of the network by setting include_top to False
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

# Freeze the layers in pre-trained model, except last 8 layers
for layer in base_model.layers[:-8]:
  layer.trainable = False

# Show the summary and read the number of trainable parameters
base_model.summary()

### Add custom layers at the end of downloaded model

In [ ]:
# (c) Extend the MobileNetV2 model

# Create a global average layer before connecting MobileNetV2 with new model
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

model = tf.keras.Sequential()
model.add(base_model)

# Extend the MobileNetV2 model with:
# 1 global average layer
# 6 hidden dense layers, with 128, 64, 128, 64, 128, 64 nodes each, and all using relu activation function
# 1 output layer with size of 10, and softmax activation function
# This configuration achieves overall accuracy of >85% on the testing dataset after training

model.add(global_average_layer)

model.add(layers.Dense(128, activation=tf.nn.relu))
model.add(layers.Dense(64, activation=tf.nn.relu))
model.add(layers.Dense(128, activation=tf.nn.relu))
model.add(layers.Dense(64, activation=tf.nn.relu))
model.add(layers.Dense(128, activation=tf.nn.relu))
model.add(layers.Dense(64, activation=tf.nn.relu))

model.add(layers.Dense(10, activation=tf.nn.softmax))




### Add loss function, compile and train the model, and check accuracy on test data

In [ ]:
# (d) Compile the neural network with:
# cross-entropy loss function
# Adamax optimizer with learning rate of 1x10^-3

model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Show the hierarchy of the model
model.summary()

In [ ]:
# Train the neural network 
# Run the stochastic gradient descent for batch size : 32 and epochs : 6
epochs = 6
batch_size = 32

hist2 = model.fit(resized_train_images, train_labels, batch_size=batch_size, epochs=epochs)

In [ ]:
test_loss, test_acc = model.evaluate(resized_test_images, test_labels, batch_size=32)

In [ ]:
print('Test accuracy:', test_acc)

In [ ]:
# Get all predictions for test data
predictions = model.predict(resized_test_images)

In [ ]:
# Code to visualize predictions
# Incorrect predictions are highlighted in red
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(resized_test_images[i], cmap=plt.cm.binary)
    predicted_label = np.argmax(predictions[i])
    true_label = test_labels[i][0]
    if predicted_label == true_label:
      color = 'green'
    else:
      color = 'red'
    plt.xlabel("{} ({})".format(class_names[predicted_label], 
                                  class_names[true_label]),
                                  color=color)

### Extra code for producing different plots

In [ ]:
#Extract the training metrics for evaluation
hist_df = pd.DataFrame(hist2.history)

epoch = [1,2,3,4,5,6]
hist_df['epoch no'] = epoch 
hist_df = hist_df.set_index('epoch no')

# Plot the result 
SAMPLE_NO = 60000

hist_arr = hist_df.to_numpy()

x = np.array([1,2,3,4,5,6])

In [ ]:
y = np.array([
     hist_arr[0][0],
     hist_arr[1][0],
     hist_arr[2][0],
     hist_arr[3][0],
     hist_arr[4][0],
     hist_arr[5][0],
    ])

plt.figure(figsize=(10,10))
plt.plot(x,y,color='green',marker='o')
plt.xlabel('Epoch Number')
plt.ylabel('Loss Function Value')
plt.title("Loss Function Value against Epoch Number")
plt.grid(True)
plt.show()


In [ ]:
# Show the overall accuracy

print('-----Final accuracy for test dataset-----')
print('overall accuracy: ' + str("{:.2f}".format(test_acc * 100)) + '\n\n')